In [33]:
import numpy as np
import pandas as pd
import csv

In [34]:
data = pd.read_csv('A:/PWR/project/AI/mushroom2/mushrooms.csv')
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [35]:
data[data['class']=="p"].shape, data[data['class']=="e"].shape

((3916, 23), (4208, 23))

In [36]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data2 = data.apply(lb.fit_transform)

In [37]:
data2.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [38]:
x = data2.drop(["class"],axis=1)
y = data2["class"].values

In [39]:
# train test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

### distance

In [40]:
def distance(a, b):
    a = np.array(a) 
    b = np.array(b)
    
    return np.linalg.norm(a - b)

### voting

In [41]:
from collections import Counter
def claspred(neighbors):
    classCounter = Counter()
    for neighbor in neighbors:
        classCounter[neighbor] += 1
    return classCounter.most_common(1)[0][0]

### find distance

with open('A:/PWR/project/AI/mushroom/barudistance.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["index_test","index_train", "distance", "class_real","class_pred"])
    for index in range(len(x_test)):
        for ind in range(len(x_train)):
            dist = distance(x_test[index:index+1],x_train[ind:ind+1])
            spamwriter.writerow([index,ind, dist, y_test[index], y_train[ind]])

In [61]:
data3 = pd.read_csv('A:/PWR/project/AI/mushroom2/barudistance.csv')
data3.head()

,index_test,index_train,distance,class_real,class_pred
0,0,0,6.633250,1,0
1,0,1,7.211103,1,0
2,0,2,13.304135,1,1
3,0,3,8.000000,1,0
4,0,4,13.190906,1,1


In [62]:
with open('A:/PWR/project/AI/mushroom2/myresult.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["index_test", "class_real","class_pred"])
    inde=0
    while inde<=1624:
        sort=data3[data3['index_test'] == inde].sort_values(by=['distance'], ascending=True)
        sort2=sort.iloc[0:10]
        clp = sort2.class_pred
        clp2 = sort2.class_real
        writer.writerow([inde, claspred(clp2),claspred(clp)])
        inde=inde+1


In [63]:
data4=pd.read_csv('A:/PWR/project/AI/mushroom2/myresult.csv')

In [64]:
data4.head()

,index_test,class_real,class_pred
0,0,1,1
1,1,1,1
2,2,0,0
3,3,1,1
4,4,1,1


In [65]:
cr=[]
crind=0
while crind<=1624:
    sort=data4[data4['index_test'] == crind]
    clp=sort.class_real.values
    cr.append(claspred(clp))
    crind=crind+1

In [66]:
cp=[]
indekss=0
while indekss<=1624:
    sort=data3[data3['index_test'] == indekss].sort_values(by=['distance'], ascending=True)
    sort2=sort.iloc[0:10]
    clp=sort2.class_pred.values
    cp.append(claspred(clp))
    indekss=indekss+1

In [67]:
from sklearn.metrics import accuracy_score
print(accuracy_score(cr, cp))

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(cr, cp))
print(classification_report(cr, cp))

1.0
[[849   0]
 [  0 776]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       849
           1       1.00      1.00      1.00       776

   micro avg       1.00      1.00      1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625

